In [1]:
import numpy as np
import pandas as pd
import pyodbc

In [2]:
# Import necessary libraries to do ML
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score,precision_score,f1_score,recall_score,confusion_matrix

In [3]:
# Connection String

serverName = "sqlserver-pw.database.windows.net" 
databaseName = "db-ai"
username = "girls"
password = "Password?"

In [4]:
# Make the conncection to our Server

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+serverName+';DATABASE='+databaseName+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [5]:
#Splitting the Dataset based on the value of BEP

query_1 = "SELECT  Canale, CodAgenzia,ClusterAgenzia, CodContratto,CategoriaCespite, Cespite, TipologiaUtenza,[DataTrasf.entr.], Anagrafica,SettoreMerceologico, ListinoProposto, [Soc.Vend.Prec.],ClusterSocVend,Localita, [Cod.ISTAT], Provincia,RegioneItaliana, Zona, ModoPagamento, RedditoTotale, Mese01,MesiBEP,MesiAttivi, UltimoReddito FROM try.DatiConsumi_ML WHERE [Soc.Vend.Prec.] is not null and BEP = 1"
query_0 = "SELECT  Canale, CodAgenzia,ClusterAgenzia, CodContratto,CategoriaCespite, Cespite, TipologiaUtenza,[DataTrasf.entr.], Anagrafica,SettoreMerceologico, ListinoProposto, [Soc.Vend.Prec.],ClusterSocVend,Localita, [Cod.ISTAT], Provincia,RegioneItaliana, Zona, ModoPagamento, RedditoTotale, Mese01,MesiBEP,MesiAttivi, UltimoReddito FROM try.DatiConsumi_ML WHERE [Soc.Vend.Prec.] is not null and BEP = 0"


In [6]:
#Load the dataset into a pandas DataFrame

df_1 = pd.read_sql(query_1, cnxn)

df_0 = pd.read_sql(query_0, cnxn)

C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\2517276147.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_1 = pd.read_sql(query_1, cnxn)
C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\2517276147.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_0 = pd.read_sql(query_0, cnxn)


In [7]:
df_1['Year_Trasf.entr.'] = pd.to_datetime(df_1['DataTrasf.entr.']).dt.year
df_1['Month_Trasf.entr.'] = pd.to_datetime(df_1['DataTrasf.entr.']).dt.month
df_1['Day_Trasf.entr.'] = pd.to_datetime(df_1['DataTrasf.entr.']).dt.day
df_1.drop(['DataTrasf.entr.'],axis=1,inplace=True) 

In [27]:
df_0['Year_Trasf.entr.'] = pd.to_datetime(df_0['DataTrasf.entr.']).dt.year
df_0['Month_Trasf.entr.'] = pd.to_datetime(df_0['DataTrasf.entr.']).dt.month
df_0['Day_Trasf.entr.'] = pd.to_datetime(df_0['DataTrasf.entr.']).dt.day
df_0.drop(['DataTrasf.entr.'],axis=1,inplace=True) 

Informazioni DataFrame BEP = 1

In [8]:
df_1.describe()

,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito,Year_Trasf.entr.,Month_Trasf.entr.,Day_Trasf.entr.
count,134868.000000,134868.000000,134868.000000,134868.000000,134868.000000,134868.000000,134868.000000,134868.000000
mean,112.176333,-95.134135,14.790929,31.940075,106.876538,2017.229424,6.283655,1.001142
std,113.113342,40.931439,7.005416,15.215921,100.019825,1.745076,3.295324,0.133394
min,0.000000,-174.200000,0.000000,1.000000,-2.500000,2015.000000,1.000000,1.000000
25%,30.010000,-120.100000,12.000000,20.000000,30.500000,2016.000000,4.000000,1.000000
50%,72.360000,-109.200000,15.000000,27.000000,72.800000,2017.000000,6.000000,1.000000
75%,155.250000,-67.700000,17.000000,42.000000,155.100000,2019.000000,9.000000,1.000000
max,591.100000,2.800000,59.000000,60.000000,499.800000,2020.000000,12.000000,26.000000


In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134868 entries, 0 to 134867
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Canale               134868 non-null  object 
 1   CodAgenzia           134868 non-null  object 
 2   ClusterAgenzia       134868 non-null  object 
 3   CodContratto         134868 non-null  object 
 4   CategoriaCespite     134868 non-null  object 
 5   Cespite              134868 non-null  object 
 6   TipologiaUtenza      134868 non-null  object 
 7   Anagrafica           134868 non-null  object 
 8   SettoreMerceologico  134868 non-null  object 
 9   ListinoProposto      134868 non-null  object 
 10  Soc.Vend.Prec.       134868 non-null  object 
 11  ClusterSocVend       134868 non-null  object 
 12  Localita             134868 non-null  object 
 13  Cod.ISTAT            134868 non-null  object 
 14  Provincia            134868 non-null  object 
 15  RegioneItaliana  

In [10]:
df_1.head(10)

,Canale,CodAgenzia,ClusterAgenzia,CodContratto,CategoriaCespite,Cespite,TipologiaUtenza,Anagrafica,SettoreMerceologico,ListinoProposto,...,Zona,ModoPagamento,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito,Year_Trasf.entr.,Month_Trasf.entr.,Day_Trasf.entr.
0,4,1,3,225678,3,3,2,62,2,6,...,2,3,218.85,-105.2,18,58,224.7,2017,1,1
1,4,1,3,225724,3,3,2,64,2,6,...,2,3,218.85,-105.2,18,58,224.7,2017,1,1
2,4,1,3,225729,2,2,1,59,2,6,...,2,1,85.35,-107.6,31,58,88.8,2017,1,1
3,4,1,3,225730,3,3,2,59,2,6,...,2,3,218.85,-105.2,18,58,224.7,2017,1,1
4,4,1,3,225731,3,3,2,49,2,6,...,2,3,218.85,-105.2,18,58,224.7,2017,1,1
5,4,1,3,136119,5,4,3,62,1,7,...,2,1,379.75,-109.2,15,60,327.9,2016,3,1
6,4,1,3,028490,5,4,3,13,1,7,...,2,3,454.12,-109.1,15,60,334.3,2015,6,1
7,4,1,3,225738,2,2,1,53,2,6,...,2,3,91.38,-107.4,30,58,95.0,2017,1,1
8,2,1,5,291309,5,4,3,35,1,7,...,2,3,149.79,-67.7,9,30,151.6,2018,4,1
9,2,1,5,291689,5,4,3,5,1,7,...,3,3,157.87,-69.2,9,31,157.6,2018,4,1


In [11]:
df_1.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\900530298.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_1.corr().style.background_gradient(cmap='coolwarm').set_precision(2)
C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\900530298.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_1.corr().style.background_gradient(cmap='coolwarm').set_precision(2)


,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito,Year_Trasf.entr.,Month_Trasf.entr.,Day_Trasf.entr.
RedditoTotale,1.00,0.04,-0.12,0.83,0.98,-0.34,-0.04,0.00
Mese01,0.04,1.00,-0.78,-0.16,0.06,0.33,0.01,-0.00
MesiBEP,-0.12,-0.78,1.00,0.30,-0.14,-0.27,-0.01,-0.00
MesiAttivi,0.83,-0.16,0.30,1.00,0.83,-0.41,-0.04,-0.00
UltimoReddito,0.98,0.06,-0.14,0.83,1.00,-0.31,-0.03,0.00
Year_Trasf.entr.,-0.34,0.33,-0.27,-0.41,-0.31,1.00,-0.11,0.00
Month_Trasf.entr.,-0.04,0.01,-0.01,-0.04,-0.03,-0.11,1.00,0.01
Day_Trasf.entr.,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.01,1.00


Informazioni DataFrame BEP = 0

In [12]:
df_0.describe()

,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito
count,339857.000000,339857.000000,339857.000000,339857.000000,339857.000000
mean,-78.425384,-118.045652,8.836475,8.837608,-78.222921
std,37.940486,25.623906,8.246860,8.248442,37.967507
min,-175.540000,-175.400000,1.000000,1.000000,-175.300000
25%,-106.800000,-126.700000,3.000000,3.000000,-106.500000
50%,-84.370000,-120.100000,6.000000,6.000000,-84.400000
75%,-47.560000,-109.200000,12.000000,12.000000,-47.300000
max,0.000000,-0.100000,55.000000,55.000000,9.200000


In [13]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339857 entries, 0 to 339856
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Canale               339857 non-null  object 
 1   CodAgenzia           339857 non-null  object 
 2   ClusterAgenzia       339857 non-null  object 
 3   CodContratto         339857 non-null  object 
 4   CategoriaCespite     339857 non-null  object 
 5   Cespite              339857 non-null  object 
 6   TipologiaUtenza      339857 non-null  object 
 7   DataTrasf.entr.      339857 non-null  object 
 8   Anagrafica           339857 non-null  object 
 9   SettoreMerceologico  339857 non-null  object 
 10  ListinoProposto      339857 non-null  object 
 11  Soc.Vend.Prec.       339857 non-null  object 
 12  ClusterSocVend       339857 non-null  object 
 13  Localita             339857 non-null  object 
 14  Cod.ISTAT            339857 non-null  object 
 15  Provincia        

In [14]:
df_0.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\663419134.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_0.corr().style.background_gradient(cmap='coolwarm').set_precision(2)
C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\663419134.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_0.corr().style.background_gradient(cmap='coolwarm').set_precision(2)


,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito
RedditoTotale,1.00,0.44,0.58,0.58,1.00
Mese01,0.44,1.00,0.08,0.08,0.44
MesiBEP,0.58,0.08,1.00,1.00,0.58
MesiAttivi,0.58,0.08,1.00,1.00,0.58
UltimoReddito,1.00,0.44,0.58,0.58,1.00


In [15]:
df_0.head(10)

,Canale,CodAgenzia,ClusterAgenzia,CodContratto,CategoriaCespite,Cespite,TipologiaUtenza,DataTrasf.entr.,Anagrafica,SettoreMerceologico,...,Cod.ISTAT,Provincia,RegioneItaliana,Zona,ModoPagamento,RedditoTotale,Mese01,MesiBEP,MesiAttivi,UltimoReddito
0,6,1,1,203252,5,4,3,2016-10-01,52,1,...,6162,76,15,3,1,-45.71,-121.7,11,11,-45.6
1,6,1,1,230585,5,4,3,2017-01-01,57,1,...,4364,22,4,3,1,-44.46,-120.2,11,11,-44.1
2,6,1,1,230586,5,4,3,2017-01-01,5,1,...,457,59,4,3,1,-44.46,-120.2,11,11,-44.1
3,6,1,1,230588,5,4,3,2017-01-01,44,1,...,433,22,4,2,1,-44.46,-120.2,11,11,-44.1
4,6,1,1,230591,5,4,3,2017-01-01,44,1,...,433,22,4,2,1,-44.46,-120.2,11,11,-44.1
5,6,1,1,230592,5,4,3,2017-01-01,66,1,...,611,23,15,3,1,-44.46,-120.2,11,11,-44.1
6,6,1,1,230603,5,4,3,2017-01-01,5,1,...,5226,46,13,3,1,-44.46,-120.2,11,11,-44.1
7,6,1,1,230605,5,4,3,2017-01-01,49,1,...,467,59,4,3,1,-44.46,-120.2,11,11,-44.1
8,6,1,1,230611,5,4,3,2017-01-01,46,1,...,436,22,4,3,1,-44.46,-120.2,11,11,-44.1
9,6,1,1,230618,5,4,3,2017-01-01,53,1,...,4543,59,4,3,1,-45.96,-121.7,11,11,-45.6


Machine Learning BEP 1

In [16]:
y_1 = df_1[['MesiAttivi', 'UltimoReddito', 'MesiBEP']]             #Targets
features_to_exclude = ['MesiAttivi', 'UltimoReddito', 'MesiBEP']
X_1 = df_1.drop(features_to_exclude, axis=1)                        #features

In [17]:
# summarize dataset
print(X_1.shape, y_1.shape)

(134868, 23) (134868, 3)


In [18]:
X_1

,Canale,CodAgenzia,ClusterAgenzia,CodContratto,CategoriaCespite,Cespite,TipologiaUtenza,Anagrafica,SettoreMerceologico,ListinoProposto,...,Cod.ISTAT,Provincia,RegioneItaliana,Zona,ModoPagamento,RedditoTotale,Mese01,Year_Trasf.entr.,Month_Trasf.entr.,Day_Trasf.entr.
0,4,1,3,225678,3,3,2,62,2,6,...,3743,5,16,2,3,218.85,-105.2,2017,1,1
1,4,1,3,225724,3,3,2,64,2,6,...,3743,5,16,2,3,218.85,-105.2,2017,1,1
2,4,1,3,225729,2,2,1,59,2,6,...,3772,5,16,2,1,85.35,-107.6,2017,1,1
3,4,1,3,225730,3,3,2,59,2,6,...,3765,5,16,2,3,218.85,-105.2,2017,1,1
4,4,1,3,225731,3,3,2,49,2,6,...,3759,5,16,2,3,218.85,-105.2,2017,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134863,1,1,5,354399,5,4,3,61,1,7,...,3743,5,16,2,1,10.27,-0.6,2019,5,1
134864,5,1,5,140122,3,3,2,53,2,9,...,3743,5,16,2,1,5.98,-40.4,2016,4,1
134865,5,1,5,244588,3,3,2,75,2,9,...,3583,34,16,1,1,4.74,-41.9,2017,4,1
134866,1,1,1,408679,5,4,3,69,1,1,...,396,15,5,3,1,0.50,0.6,2020,1,1


In [19]:
y_1

,MesiAttivi,UltimoReddito,MesiBEP
0,58,224.7,18
1,58,224.7,18
2,58,88.8,31
3,58,224.7,18
4,58,224.7,18
...,...,...,...
134863,2,7.0,1
134864,7,6.2,6
134865,7,4.7,6
134866,1,0.6,0


In [20]:
y_1.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_5108\3696670376.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  y_1.corr().style.background_gradient(cmap='coolwarm').set_precision(2)


,MesiAttivi,UltimoReddito,MesiBEP
MesiAttivi,1.00,0.83,0.30
UltimoReddito,0.83,1.00,-0.14
MesiBEP,0.30,-0.14,1.00


In [21]:
# Define and fit models
models = {
    'LinearRegression' : LinearRegression(),
    'Ridge' :   Ridge(solver='svd'),
    'MultiTarget_GradientBoostingRegressor' : MultiOutputRegressor(GradientBoostingRegressor()),
    'Chain_GradientBoostingRegressor' : RegressorChain(GradientBoostingRegressor(),verbose=True),
    'MultiTarget_Lasso' :   MultiOutputRegressor(Lasso(alpha=0.2)),
    'Chain_Lasso'   :   RegressorChain(Lasso(alpha=0.2),verbose=True),
    'MultiTarget_LassoRN' :   MultiOutputRegressor(Lasso(alpha=0.2,selection='random')),
    'Chain_LassoRN'   :   RegressorChain(Lasso(alpha=0.2,selection='random'),verbose=True),
    'DecisionTreeRegressor' : DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(algorithm='auto')
}

In [22]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.2, random_state=42)


In [23]:
# Evaluate and make predictions for each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #accuracy = accuracy_score(y_test, y_pred)
   #precision = precision_score(y_test, y_pred)
   #recall = recall_score(y_test, y_pred)
   #f1 = f1_score(y_test, y_pred)
    #cm = confusion_matrix(y_test, y_pred)

    print(f"{name} Mean Squared Error: {mse}")
    print(f"{name} R2 score: {r2}")
    #print(f"{name} Accuracy score: {accuracy}")
    #print(f'Confusion Matrix:\n{cm}')


    #ereg = VotingRegressor([{name},{model}])
    #ereg.fit(X_train,y_train)

LinearRegression Mean Squared Error: 103.5132723458667
LinearRegression R2 score: 0.8901482858764632
Ridge Mean Squared Error: 103.51327557785267
Ridge R2 score: 0.8901478411099698
MultiTarget_GradientBoostingRegressor Mean Squared Error: 8.209284169869326
MultiTarget_GradientBoostingRegressor R2 score: 0.9901035477917688
[Chain] ................... (1 of 3) Processing order 0, total=  19.7s
[Chain] ................... (2 of 3) Processing order 1, total=  20.6s
[Chain] ................... (3 of 3) Processing order 2, total=  22.1s
Chain_GradientBoostingRegressor Mean Squared Error: 10.139356124102223
Chain_GradientBoostingRegressor R2 score: 0.9882928687444107
MultiTarget_Lasso Mean Squared Error: 106.04538043001446
MultiTarget_Lasso R2 score: 0.8707145060463981
[Chain] ................... (1 of 3) Processing order 0, total=   3.0s
[Chain] ................... (2 of 3) Processing order 1, total=   4.0s
[Chain] ................... (3 of 3) Processing order 2, total=   1.9s
Chain_Lasso Me

Machine Learning BEP 0

In [28]:
y_0 = df_0[['MesiAttivi', 'UltimoReddito', 'MesiBEP']]             #Targets
features_to_exclude = ['MesiAttivi', 'UltimoReddito', 'MesiBEP']
X_0 = df_0.drop(features_to_exclude, axis=1)     

In [29]:
X0_train, X0_test, y0_train, y0_test = train_test_split(X_0, y_0, test_size=0.2, random_state=42)

In [30]:
# Evaluate and make predictions for each model
for name, model in models.items():
    model.fit(X0_train, y0_train)
    y0_pred = model.predict(X0_test)
    mse = mean_squared_error(y0_test, y0_pred)
    r2 = r2_score(y0_test, y0_pred)
    #accuracy = accuracy_score(y_test, y_pred)
   #precision = precision_score(y_test, y_pred)
   #recall = recall_score(y_test, y_pred)
   #f1 = f1_score(y_test, y_pred)
    #cm = confusion_matrix(y_test, y_pred)

    print(f"{name} Mean Squared Error: {mse}")
    print(f"{name} R2 score: {r2}")
    #print(f"{name} Accuracy score: {accuracy}")
    #print(f'Confusion Matrix:\n{cm}')


    #ereg = VotingRegressor([{name},{model}])
    #ereg.fit(X_train,y_train)

LinearRegression Mean Squared Error: 20.206915251232385
LinearRegression R2 score: 0.7031149323911686
Ridge Mean Squared Error: 20.206903390325518
Ridge R2 score: 0.7031151075658263
MultiTarget_GradientBoostingRegressor Mean Squared Error: 3.5277218106102755
MultiTarget_GradientBoostingRegressor R2 score: 0.9497056825123465
[Chain] ................... (1 of 3) Processing order 0, total= 1.4min
[Chain] ................... (2 of 3) Processing order 1, total= 2.0min
[Chain] ................... (3 of 3) Processing order 2, total= 2.1min
Chain_GradientBoostingRegressor Mean Squared Error: 3.564504000373519
Chain_GradientBoostingRegressor R2 score: 0.9491708244822753
MultiTarget_Lasso Mean Squared Error: 20.96154926948017
MultiTarget_Lasso R2 score: 0.6919774654369232
[Chain] ................... (1 of 3) Processing order 0, total=   6.4s
[Chain] ................... (2 of 3) Processing order 1, total=   0.7s
[Chain] ................... (3 of 3) Processing order 2, total=   5.1s
Chain_Lasso Me